# Bringing It All Together: Complete Economic Analysis

Time to combine everything we've got - FRED economic data, NBER recession dates, and consumer sentiment - into one comprehensive dataset. This is where we'll see the full picture of how different economic signals work together to predict downturns.

In [ ]:
# Import notebook utilities
from notebook_utils import init_notebook, load_data, display_data_info, save_figure
import os
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display

# Initialize notebook environment
init_notebook()

# Import from econ_downturn package
from econ_downturn import (
    engineer_features, normalize_data, apply_mda, create_discriminant_time_series,
    plot_indicator_with_recessions, plot_correlation_matrix,
    plot_feature_importance, plot_discriminant_time_series,
    plot_sentiment_vs_indicator, plot_sentiment_correlation_matrix
)

## Loading All Our Data Sources

Let's pull together data from all our sources - FRED economic indicators, NBER recession dates, and University of Michigan consumer sentiment. This gives us the complete picture.

In [ ]:
# Load all data using the utility function
merged_data = load_data(use_cached=False)  # Force reload from original sources

# Display information about the dataset
display_data_info(merged_data)

## Exploring Consumer Sentiment Patterns

Now let's see how consumer sentiment behaves around recessions. Does it drop before economic downturns? How does it relate to other indicators like unemployment and GDP?

In [ ]:
# Plot consumer sentiment over time with recession periods
fig = plot_indicator_with_recessions(
    merged_data, 
    'SENTIMENT', 
    title='Consumer Sentiment with Recession Periods'
)
plt.show()
save_figure(fig, "consumer_sentiment.png")

In [ ]:
# Plot consumer sentiment vs unemployment rate
fig = plot_sentiment_vs_indicator(
    merged_data,
    sentiment_col='SENTIMENT',
    indicator_col='UNEMPLOYMENT'
)
plt.show()
save_figure(fig, "sentiment_vs_unemployment.png")

In [ ]:
# Plot consumer sentiment vs GDP growth
fig = plot_sentiment_vs_indicator(
    merged_data,
    sentiment_col='SENTIMENT',
    indicator_col='GDP'
)
plt.show()
save_figure(fig, "sentiment_vs_gdp.png")

In [ ]:
# Plot correlations between consumer sentiment and economic indicators
fig = plot_sentiment_correlation_matrix(
    merged_data,
    sentiment_cols=['SENTIMENT'],
    top_n=10
)
plt.show()
save_figure(fig, "sentiment_correlation_matrix.png")

## Engineering Features with Sentiment Data

Time to create some smart features using our combined dataset. We'll build lag variables and transformations that capture how sentiment and economic indicators interact over time.

In [ ]:
# Handle missing values and create lag variables
data_with_features = engineer_features(merged_data)

print(f"Data with features shape: {data_with_features.shape}")
print(f"Number of features: {data_with_features.shape[1]}")

# Save the dataset with features
from econ_downturn import get_data_paths
data_paths = get_data_paths()
output_dir = data_paths['processed_dir']
os.makedirs(output_dir, exist_ok=True)
features_path = os.path.join(output_dir, 'data_with_features_and_sentiment.csv')
data_with_features.to_csv(features_path)
print(f"Saved dataset with features to {features_path}")

## Normalizing for Fair Comparison

Different indicators have different scales, so let's normalize everything to ensure our model treats all features equally.

In [ ]:
# Normalize the data
data_normalized, scaler = normalize_data(data_with_features)

print(f"Normalized data shape: {data_normalized.shape}")

# Save the normalized dataset
normalized_path = os.path.join(output_dir, 'data_normalized_with_sentiment.csv')
data_normalized.to_csv(normalized_path)
print(f"Saved normalized dataset to {normalized_path}")

## Testing Our Enhanced Model

The moment of truth - does adding consumer sentiment actually make our recession prediction model better? Let's run a quick test and see.

In [ ]:
# Separate features and target
X = data_normalized.drop(columns=['recession'])
y = data_normalized['recession']

# Apply MDA
mda_results = apply_mda(X, y)

# Print model performance metrics
print(f"Accuracy: {mda_results['accuracy']:.4f}")
print("\nConfusion Matrix:")
print(mda_results['conf_matrix'])
print("\nClassification Report:")
print(mda_results['class_report'])
print(f"\nCross-Validation Scores: {mda_results['cv_scores']}")
print(f"Mean CV Score: {mda_results['cv_scores'].mean():.4f}")

In [ ]:
# Plot feature importances
if mda_results['feature_importance'] is not None:
    fig = plot_feature_importance(mda_results['feature_importance'])
    plt.show()
    save_figure(fig, "feature_importance_with_sentiment.png")

In [ ]:
# Create and plot discriminant time series
discriminant_df = create_discriminant_time_series(
    mda_results['model'], X, y
)

fig = plot_discriminant_time_series(discriminant_df)
plt.show()
save_figure(fig, "discriminant_time_series_with_sentiment.png")